<a href="https://colab.research.google.com/github/organisciak/Scripting-Course/blob/master/labs/03-joins-like-methods-lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab Worksheet 3

*Reminder - save your work. Go to* File > Save a Copy in Drive *to ensure that you have your work saved.*

Hopefully by this week you're getting comfortable with Python and SQL, and understand the differences between the two. If not, ask questions and refer to our prior notes.

This will be the last week where we talk about fundamental skills for Python and SQL - starting next week, we're putting them to use!

SQL Skills

- `ORDER BY`
- `LIMIT`
- `AND` and `OR`
- `LIKE` clause for `WHERE`
- `JOIN`

## Lab preparation

Download [circulation.sqlite](https://github.com/organisciak/Scripting-Course/blob/master/data/circulation.sqlite?raw=true) and place in the same folder as this lab. You can do it manually - download in your browser, open the Files tab on the right, and drag it there - or you can run this command:

In [0]:
#@markdown *Run me to download our circulation data sqlite database*
!wget -nv https://github.com/organisciak/Scripting-Course/blob/master/data/circulation.sqlite?raw=true -O circulation.sqlite

2020-04-14 23:15:02 URL:https://raw.githubusercontent.com/organisciak/Scripting-Course/master/data/circulation.sqlite [577536/577536] -> "circulation.sqlite" [1]


In [0]:
#@markdown *Lab Checking Code*: This lab has some answers that it provided - run this code to download them.
!python -m pip install -q git+https://github.com/organisciak/Scripting-Course-Grading.git@master
from scripting_grading import grading, lab3
import pandas as pd

Remember that in Colab, the files that you save are temporary - they'll disappear
after a few hours.

Once you have your database, connect to it in this way:

In [0]:
%load_ext sql
%sql sqlite:///circulation.sqlite

'Connected: @circulation.sqlite'

This is a generated dataset of library circulation information, with three tables: `patrons`, `books`, and `circulation`.

## SQL

Last week, we reviewed `CREATE TABLE`, `INSERT`, and `SELECT`. Today, we'll cover a bit more about clauses that allow us to modify how we SELECT results.


## `LIMIT` and `ORDER BY`

`LIMIT` can limit the number of results that `SELECT` returns. This is especially useful if there are many, many matches, and you only want to see some of them.

`ORDER BY` allows you to specify a column to sort by, and whether it is sorted in ascending order (`ASC`) or descending order (`DESC`).

These clauses are usually at the end of the statement - if using both the `LIMIT` should go later than `ORDER BY`. The best way to understand them is by example:

*Example: Select 5 records from the patrons table.*

In [0]:
%%sql
SELECT * FROM patrons
    LIMIT 5;

 * sqlite:///circulation.sqlite
Done.


id,name,age,email,zipcode,city,activity
1,Jeremy Mata,30,timothymoore@gmail.com,52022,Thomasmouth,9
2,Daniel Garcia,55,mcdanielwilliam@sanchez-herrera.biz,09529,Deborahbury,3
3,Warren Green,17,janice81@flores.com,55036,Michelefurt,1
4,Erika Smith,4,tnichols@gregory.com,11235,Port Stephanie,3
5,Matthew Johnson,22,christina30@manning.info,52521,Jamesfort,4


(*If this cell doesn't work, it means you didn't download *circulation.sqlite*)

*Example: Select the five oldest patrons.*

In [0]:
%%sql
SELECT * FROM patrons
    ORDER BY age DESC
    LIMIT 5;

 * sqlite:///circulation.sqlite
Done.


id,name,age,email,zipcode,city,activity
516,Latoya Clark,82,lanceosborne@hotmail.com,79851,Deborahbury,9
911,Jackie Arias,80,sandra83@ramirez-wheeler.com,53858,Michelefurt,9
89,Jennifer Bradley,79,jacquelinenoble@williams-ayala.com,63196,Port Stephanie,4
385,Shannon Smith,79,christinamullen@yahoo.com,24061,Walkerview,0
238,Christopher James,78,cookchristopher@yahoo.com,71849,Cooperton,4


## More on `WHERE`

### `OR`

Multiple conditions can be combined with `AND`; e.g. `WHERE age > 10 AND first_name == 'Jack'`. It is also possible to say 'one of these conditions' by using `OR`. For example,

```sql
SELECT * FROM people
    WHERE age > 10 OR first_name == 'Jack';
```

This will return all records where the person's age is >10 and all records where the first name is Jack.

You can avoid ambiguity by using parentheses. The following is functionally identical, but slightly easier to read and tougher to accidentally do something you didn't intend:

```sql
SELECT * FROM people
    WHERE (age > 10) OR (first_name == 'Jack');
```

### `LIKE`

In addition to logical operators - e.g. `WHERE age > 10` or `WHERE first_name == 'Jack'` - you can also do a partial match on a string with `LIKE`.

So, `WHERE first_name LIKE 'Ja%'` will match first names like James, Jack, Jan.

`LIKE` tells SQL to look for special characters in the string. There are two special characters that you should know:

- `%`, like in the example above, matched any number of random characters
- `_` matches a single random character.

*Example: match names that start with Jacob, followed by any other characters:*

In [0]:
%%sql
SELECT * FROM patrons
    WHERE name LIKE 'Jacob%';

 * sqlite:///circulation.sqlite
Done.


id,name,age,email,zipcode,city,activity
224,Jacob Conrad,34,racheldurham@king-downs.com,48807,New Hannah,5
248,Jacob Rivas,32,larry34@castillo-gibson.net,48618,East Pamelaburgh,8
287,Jacob Ross,53,phillipsvalerie@hudson.biz,98672,Cooperton,6
417,Jacob Palmer,44,willie34@hernandez-king.com,88821,Loganchester,1
428,Jacob Reid,34,dustin32@lawson-miller.biz,61655,Colechester,2
943,Jacob Garcia,50,vsalinas@gmail.com,62186,Romerofort,5


*Example: [**Doing it wrong**] - Search for @gmail emails. Problem - this searches for strings starting with @gmail, but you need to say that there can be other characters before `@`*

In [0]:
%%sql
SELECT * FROM patrons
    WHERE email LIKE '@gmail.com';

 * sqlite:///circulation.sqlite
Done.


id,name,age,email,zipcode,city,activity


*Example: [**Correct**] - Search for @gmail emails.*

In [0]:
%%sql
SELECT * FROM patrons
    WHERE email LIKE '%@gmail.com';

 * sqlite:///circulation.sqlite
Done.


id,name,age,email,zipcode,city,activity
1,Jeremy Mata,30,timothymoore@gmail.com,52022,Thomasmouth,9
15,Brian Nixon,24,combslee@gmail.com,76117,Rothton,9
19,Shelly Roth,46,carloswalker@gmail.com,03442,Austinshire,6
21,James Arnold,22,faithchavez@gmail.com,15431,South Danielport,5
22,Jennifer Evans,49,ijackson@gmail.com,69375,East Pamelaburgh,6
23,Brittany Coleman,23,watsonscott@gmail.com,34912,North Stephenfort,7
29,Alyssa Williams DVM,33,tonya62@gmail.com,55420,Normanville,4
32,Tiffany Brooks,31,jeffreyburgess@gmail.com,10722,South Matthew,7
34,Victoria Carpenter,28,victoria72@gmail.com,20246,Thomasmouth,5
35,Allison Payne,31,thomas19@gmail.com,23188,Lake Michaelfort,4


*Example: Find names that start with a two-character prefix, followed by a period and space*

In [0]:
%%sql
SELECT * FROM patrons
    WHERE name LIKE '__. %';

 * sqlite:///circulation.sqlite
Done.


id,name,age,email,zipcode,city,activity
66,Mr. Andrew Hahn,23,showard@lee.com,50404,Colechester,0
85,Dr. Sarah Yates,21,angelasmith@hotmail.com,68316,Jamesfort,5
97,Mr. Jesse Thomas,36,ewright@gmail.com,64152,New Terri,5
164,Mr. Nathan Murphy,46,sharon88@miller.com,87025,New Terri,8
179,Mr. Kevin Lewis,32,zamoratiffany@yahoo.com,53680,Lake Marissa,7
372,Dr. Brian Chavez,28,emilyshah@erickson-ellison.com,09276,Lake Michaelfort,4
386,Mr. Philip Allen,49,djohnson@hotmail.com,06663,Jaredville,1
462,Mr. Gabriel Martin,34,jennifer02@murphy.biz,48427,Colechester,6
574,Ms. Cheryl Ruiz,31,sgrimes@morales.com,02481,North Stephenfort,9
630,Ms. Erin Walsh DDS,8,richardlowery@duncan.biz,49044,Marystad,9


### SQL Questions 1

**Q1**: (*4pts*) Write the SQL to select the first 10 records of the `books` table.

In [0]:
# Answer-Q1
%%sql


In [0]:
#@markdown *Run this cell to see the expected output*
pd.DataFrame(lab3.L3_Q1_Answers.output)

**Q2**: (*4pts*) Write the SQL to select the `patrons` that are equal to or over 65 years old.

In [0]:
# Answer-Q2
%%sql 



In [0]:
#@markdown *Run this cell to see the expected output*
pd.DataFrame(lab3.L3_Q2_Answers.output)

**Q3**: Write the SQL to select any patrons that are 8 years old and have the zip code 49044.

In [0]:
# Answer-Q3
%%sql 


In [0]:
#@markdown *Run this cell to see the expected output*
print("Impressive Dentist!")
pd.DataFrame(lab3.L3_Q3_Answers.output)

In [0]:
#@markdown **Q4**: (*4pts*) What is the id of the most recently returned book listed in the `circulation` table?

q4_answer = 0 #@param {type:'integer'}
#@markdown Tip: you'll want to first check the columns in the table to know what to sort on.

In [0]:
#@markdown **Q5**: (*4pts*) How many records are listed in `books` with `George Orwell` as the author?

q5_answer = 0 #@param {type:'integer'}

In [0]:
#@markdown **Q6**: Complete the `WHERE .. LIKE` search for the following searches against the `books` table:

#@markdown  - a) (*4pts*) Written by an author with the middle initial `C`, like in `Arthur C. Clarke`: `WHERE author LIKE ....`
q_6a_answer = '' #@param {type:'string'}
#@markdown  - b) (*4pts*) Title starting with the word `The`: `WHERE title LIKE ...`

q_6b_answer = '' #@param {type:'string'}

#@markdown  - c) (*5pts*) Written by authors named like `Tim` or `Tom` (starts with a `T`, three characters, ends with an `m`): `WHERE author LIKE ...`
q_6c_answer = '' #@param {type:'string'}

**Q7**: (*5pts*) Write the SQL for the last question, matching *only* Tim or Tom (e.g. no 'Tam'). Tip: Don't get fancy, try an `or` statement.

In [0]:
# Answer-Q7
%%sql


**Q8**: (*5pts*) How would you retrieve emails from `.org` domains? Write the SQL.

In [0]:
# Answer-Q8
%%sql


**Q9**: (*5pts*) Write the SQL to retrieve any people whose last name starts with `Ta`. Hint: the results should look like:

![](https://github.com/organisciak/Scripting-Course/blob/master/images/sql-select-ta.png?raw=1)

In [0]:
# Answer-Q9
%%sql


## SQL Joins

Review the lecture materials from this week to learn about JOINs. e.g.

In [0]:
%%sql
SELECT books.title, patrons.name, patrons.age, circulation.checkout_time FROM circulation
    JOIN books ON (circulation.book_id == books.id)
    JOIN patrons ON (circulation.patron_id == patrons.id)
LIMIT 10;

 * sqlite:///circulation.sqlite
Done.


title,name,age,checkout_time
The Curious Incident of the Dog in the Night-time,Michelle Wood,21,2017-05-11 16:53:33.000000
The Catcher in the Rye: Annotations and Study Aids,Joseph Santiago,21,2018-01-05 13:07:20.000000
"Harry Potter ve Sırlar Odası (Harry Potter, #2)",James Harris,54,2018-01-29 22:25:54.000000
"Harry Potter y el cáliz de fuego (Harry Potter, #4)",Felicia Schneider,21,2017-06-21 07:27:02.000000
Brave New World / Brave New World Revisited,Jason Brown,35,2017-08-29 16:19:26.000000
"Harry Potter et le Prisonnier d'Azkaban (Harry Potter, #3)",Anthony Mann,30,2017-08-13 15:10:10.000000
"Outlander (Outlander, #1)",Kayla Peterson,46,2017-09-26 11:38:24.000000
魔戒前傳：哈比人歷險記,Jeffrey Jimenez,22,2017-09-09 16:34:28.000000
The Time Machine: An Invention,Nathan Dalton,22,2017-10-16 18:47:49.000000
Lord Of The Flies,Gary Daugherty,26,2017-08-18 18:56:03.000000


## Questions:

Consider the following two tables: `Pets` and `Owners`. We don't have this database, but based on the structure, you should be able to answer the questions.

![](https://github.com/organisciak/Scripting-Course/blob/master/images/pets-table.png?raw=1)

**Q10**: (*7pts*) Write the SQL to retrieve the email addresses for all pets where the species is 'cat'.

*Because we're not working with a real database, you  won't actually run it (there's nothing to run against!) - just correct and finish what I put in between the triple-quotes below*:

In [0]:
q10_answer = '''
SELECT something FROM somewhere
WHERE something

'''

**Q11**: (*5pts*) Write the SQL to retrieve all the owners with a missing (i.e. null) email.

Consider the following tables for online comments: `Comments` and `Users`. Tip: remember `IS NULL` from the slides!

![](https://github.com/organisciak/Scripting-Course/blob/master/images/comment-er.png?raw=1)

In [0]:
q11_answer = '''
SELECT something FROM somewhere
WHERE something

'''


**Q12**: (*7pts*) Some users are banned in a way that they can see their own comments, but nobody else can see their comments. This is signified with a `TRUE` in the shadow_ban field. Write the SQL to retrieve all comments with information on whether the comment is shadow banned.

In [0]:
q12_answer = '''
SELECT something FROM somewhere
WHERE something

'''

In [0]:
#@markdown **Q13**: Multiple Choice. Sometimes, a user deletes their account, but the comment is in the system. Assume `comments` is the left table.

#@markdown - 1) (*3pts*) What type of JOIN is necessary to retrieve all the comments, with user names and images if available?
q13a_answer = "" #@param ["", "INNER JOIN", "LEFT OUTER JOIN", "FULL OUTER JOIN", "Not possible."]
#@markdown - 2) (*3pts*) What type of JOIN is necessary to retrieve all the comments by active users, with user names included?
q13b_answer = "" #@param ["", "INNER JOIN", "LEFT OUTER JOIN", "FULL OUTER JOIN", "Not possible."]
#@markdown - 3) (*4pts*) What type of JOIN is necessary to retrieve all the active and deleted comments by a user?
q13c_answer = "" #@param ["", "INNER JOIN", "LEFT OUTER JOIN", "FULL OUTER JOIN", "Not possible."]

## Python

### *methods* and *functions*

In Python and most programming languages, it is possible to create a 'recipe' of multiple instructions that you may want to repeat.

You can recognize a function by its parentheses. We've seen one function already:

In [0]:
print()

Of course, running the `print` function alone isn't interesting - we want to tell it *what* to print! This brings us to an important feature of functions: *arguments*. Functions aren't simply a recipe, bundling the same lines of code - they can be given arguments to modify the code in some way.

For `print`, the first argument is a value: _what is it that you want to print?_ Very basically:

In [0]:
print('test')

test


Recall from last week that we can look up code documentation by running code with a question mark at the start. So if you run:

`?print`

You'll see:

```
Docstring:
print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)

Prints the values to a stream, or to sys.stdout by default.
Optional keyword arguments:
file:  a file-like object (stream); defaults to the current sys.stdout.
sep:   string inserted between values, default a space.
end:   string appended after the last value, default a newline.
flush: whether to forcibly flush the stream.
```

There are more arguments that you can supply! 

First, you can provide *multiple* values:

In [0]:
print("Hello", "World")

Hello World


Note that there are some weird looking arguments, like `sep=' '` and `end='\n'`.

Those are _named arguments_: you can change the argument by referring to it by name. They also have a default: so if you don't tell `print` what the separator between values (`sep`) is, it will simply be a `' '` (string comprised of just one space character).

When we ran `print("Hello", "World")`, it put a space between the two worlds because that's the default. To change what's put between the values, consider this example and it's output:

In [0]:
print("Hello", "World", sep="#")

Hello#World


Some objects in Python have build-in functions, called `methods`. These look the same, but are run from the object, with a period denoting that 'this method belongs to this object'.

We saw this briefly in last week's lab, when we used code autocomplete to see what a string can do. For example, we can use the `upper` method of strings to make them uppercase:

In [0]:
teststring = "Hello world"
teststring.upper()

'HELLO WORLD'

Or the `replace` method to replace parts of the string with something else, just like Find & Replace in many programs:

In [0]:
teststring.replace('Hello', 'Goodbye')

'Goodbye world'

### Tip:

Think about what type of object is returned by a method like `teststring.replace()`. It returns a new string, with your replacements. That new string has the same methods, so you could 'chain' multiple methods:

In [0]:
teststring.replace('Hello', 'Goodbye').upper()

'GOODBYE WORLD'

This only works because the result of `replace` was a string. The important distinction: `upper()` is not run on the output of `teststring.replace`, not on `teststring` itself!

When in doubt, set objects to multiple variables on multiple lines, though chaining will be useful for multi-step analyses later in the course. This is the more clear version of the above:

In [0]:
replaced = teststring.replace('Hello', 'Goodbye')
uppercased = replaced.upper()
uppercased

'GOODBYE WORLD'

### Import Statements

In week 1, we discussed 'libraries': importable collections of code. These are tools that make your life easier. To import a library, you write:

`import libraryname`

Then you can use the `libraryname` code. 

For a real example: we'll be using Pandas a lot in this class, which you can import as `import pandas`. Since we'll be refering to it a *lot*, we can rename it to something shorter by saying 'import ... as ...'. The pandas convention is to import it with the name `pd`, like this:

In [0]:
import pandas as pd

Did it work for you? Try tab auto-complete to see the methods it offers:

![](https://github.com/organisciak/Scripting-Course/blob/master/images/pd-tab.png?raw=1)

## Questions

We'll practice importing, using methods, and looking up documentation.

**Q14**: (*7pts*) Write the Python code to do two things: import the Pandas library, then use the `read_csv` method from the library to load the following data: https://raw.githubusercontent.com/organisciak/Scripting-Course/master/data/cat_data.tsv. Note that it's separated by tabs (represent in code by a special character written as '\t') rather than commas. Set the resulting output to a variable, `data`.

Tip: Tinker, read the documentation, and don't worry about breaking stuff: you can try again!

In [0]:
# Answer-Q14


**Q15**: (*4pts*) What's the code for getting the means of all the columns for `data`? It's a method - see if autocomplete can help find it.

`data` is a table-like structure for cat measurements, listing sex, body weight (`Bwt`), and heart weight (`Hwt`). Inspect the data to see what it looks like, and compare that to the following code:

In [0]:
# Answer Q15


In [0]:
#@markdown **Q16**: (*4pts*) What does this code do:

#@markdown ```
#@markdown data[:5]
#@markdown ```

q16_answer = "" #@param {type:'string'}

In [0]:
#@markdown **Q17**: (*4pts*) What does this code do:

#@markdown ```
#@markdown data[['Sex', 'Bwt']]
#@markdown ```

q17_answer = "" #@param {type:'string'}

In [0]:
#@markdown **Q18**: (*4pts*) What does this code do:

#@markdown ```
#@markdown data.query('Hwt > 13')
#@markdown ```

q18_answer = "" #@param {type:'string'}

# Submission Instructions

In [0]:
#@markdown ### First, Enter your name for grading
my_name = "" #@param { type:'string' }

#@markdown _Have you saved your work for yourself? Don't forget to Save a Copy in Drive so that you have your progress._

In [0]:
#@markdown ### Second, check your work:

#@markdown - have you answered all the questions?
#@markdown     - Some answers can be checked automatically - just run this cell.
#@markdown - Does this notebook run from top to bottom?
#@markdown     - Go to "Runtime > Restart and run all..." to check. Do all the cells run, to the very bottom, or is there a cell in the middle with an error?
#@markdown - Have you completed all the answers where you entered code, keeping the `# Answer-Qx` line at the start of those cells?

#@markdown *A lab that the professor has to fix manually will lose 10pts - run the checks!*

#@markdown ### Finally, submit it.

#@markdown - Download the file with "File > Download .ipynb" and submit it to the Canvas assignment page.